In [3]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
sys.path.append('../..')
sys.path.append('..')
import scripts.get_metadata_theses_beautiful_soup as get_metadata_theses_beautiful_soup
import scripts.get_url_theses_selenium as get_url_theses_selenium
import scripts.get_vector_store as get_vector_store
import scripts.utilities_database as utilities_database
import scripts.RAG as RAG



/Users/avicenne/anaconda3/envs/ultra/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Reset excel

In [4]:
df = pd.read_excel("./static/excel/df_query.xlsx")
df = df.drop(df.columns, axis=1)
df.to_excel("./static/excel/df_query.xlsx", index=False)

In [116]:

response = requests.get("https://theses.fr/2024UPASL053")
soup     = BeautifulSoup(response.text, 'html.parser')
soup.find_all('tr', {'data-v-276fb210': True})

[<tr class="table-rows" data-v-276fb210=""><td data-v-276fb210="" role="rowheader"><strong data-v-276fb210="">Auteur / Autrice : </strong></td><td data-v-276fb210="" role="list"><!--[--><!--[--><a class="clickable orange" data-v-276fb210="" href="/280557515" role="listitem">Zhenrui Pan</a><!-- --><!--]--><!--]--></td></tr>,
 <tr class="table-rows" data-v-276fb210=""><td data-v-276fb210="" role="rowheader"><strong data-v-276fb210="">Direction : </strong></td><td data-v-276fb210="" role="list"><!--[--><!--[--><a class="clickable lightblue" data-v-276fb210="" href="/175378150" role="listitem">Svetlana Dokudovskaya</a><span data-v-276fb210="">, </span><!--]--><!--[--><a class="clickable lightblue" data-v-276fb210="" href="/138463794" role="listitem">Yegor Vassetzky</a><!-- --><!--]--><!--]--></td></tr>,
 <tr class="table-rows" data-v-276fb210=""><td data-v-276fb210="" role="rowheader"><strong data-v-276fb210="">Type : </strong></td><td data-v-276fb210=""><span data-v-276fb210="">Thèse de d

In [123]:
response = requests.get("https://theses.fr/2024UPASL053")
soup     = BeautifulSoup(response.text, 'html.parser')
soup.find_all('tr', {'data-v-276fb210': True})
dict_infos = {}
for i in soup.find_all('tr', {'data-v-276fb210': True}):
    infos = i.text.split(":")
    print(infos)
    dict_infos[infos[0].replace("\xa0", "")] = infos[-1].replace("\xa0", "")
    
print(dict_infos)

['Auteur / Autrice\xa0', '\xa0Zhenrui Pan']
['Direction\xa0', '\xa0Svetlana Dokudovskaya,\xa0Yegor Vassetzky']
['Type\xa0', '\xa0Thèse de doctorat']
['Discipline(s)\xa0', '\xa0Sciences du Cancer']
['Date\xa0', '\xa0Soutenance le  20/09/2024']
['Etablissement(s)\xa0', '\xa0université Paris-Saclay']
['Ecole(s) doctorale(s)\xa0', '\xa0École doctorale Cancérologie ', ' biologie-médecine-santé (Villejuif, Val-de-Marne ; 2015-....)']
['Partenaire(s) de recherche\xa0', '\xa0Laboratoire ', " Aspects métaboliques et systémiques de l'oncogénèse pour de nouvelles approches thérapeutiques (Villejuif, Val-de-Marne ; 2020-....) "]
['référent ', ' Université Paris-Saclay. Faculté de médecine (Le Kremlin-Bicêtre, Val-de-Marne ; 2020-....) ']
['graduate school ', ' Université Paris-Saclay. Graduate School Life Sciences and Health (2020-….) ']
['Jury\xa0', '\xa0Président / Présidente ', ' Alexander Ishchenko']
['Examinateurs / Examinatrices ', ' Eric Julien, Nathalie Mazure, Maria Castedo, Pierre-Emmanu

In [125]:
d = {'Auteur / Autrice': 'Zhenrui Pan', 'Direction': 'Svetlana Dokudovskaya,Yegor Vassetzky', 'Type': 'Thèse de doctorat', 'Discipline(s)': 'Sciences du Cancer', 'Date': 'Soutenance le  20/09/2024', 'Etablissement(s)': 'université Paris-Saclay', 'Ecole(s) doctorale(s)': ' biologie-médecine-santé (Villejuif, Val-de-Marne ; 2015-....)', 'Partenaire(s) de recherche': " Aspects métaboliques et systémiques de l'oncogénèse pour de nouvelles approches thérapeutiques (Villejuif, Val-de-Marne ; 2020-....) ", 'référent ': ' Université Paris-Saclay. Faculté de médecine (Le Kremlin-Bicêtre, Val-de-Marne ; 2020-....) ', 'graduate school ': ' Université Paris-Saclay. Graduate School Life Sciences and Health (2020-….) ', 'Jury': ' Alexander Ishchenko', 'Examinateurs / Examinatrices ': ' Eric Julien, Nathalie Mazure, Maria Castedo, Pierre-Emmanuel Joubert, Kristine Schauer', 'Rapporteurs / Rapporteuses ': ' Eric Julien, Nathalie Mazure'}
pd.DataFrame([d])

,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,référent,graduate school,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses
0,Zhenrui Pan,"Svetlana Dokudovskaya,Yegor Vassetzky",Thèse de doctorat,Sciences du Cancer,Soutenance le 20/09/2024,université Paris-Saclay,"biologie-médecine-santé (Villejuif, Val-de-Ma...",Aspects métaboliques et systémiques de l'onco...,Université Paris-Saclay. Faculté de médecine ...,Université Paris-Saclay. Graduate School Life...,Alexander Ishchenko,"Eric Julien, Nathalie Mazure, Maria Castedo, ...","Eric Julien, Nathalie Mazure"


### Get url theses

In [133]:
list_url, _ = get_url_theses_selenium.get_all_url_theses("water")

Résultat du parsing : ParseResult(scheme='https', netloc='theses.fr', path='/resultats', params='', query='q=water&page=1&nb=10&tri=pertinence&domaine=theses', fragment='')



## get metadata

In [139]:
df_metadata = get_metadata_theses_beautiful_soup.get_all_metadata_theses_bs(list_url)

100%|██████████| 10/10 [00:10<00:00,  1.04s/it]


In [143]:
df_metadata.head(2)

,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses,title,content,url_these,référent,Établissement co-accrédité
0,Nicole Haroun,"Jean-François Landrier,Lourdes Mounien",Thèse de doctorat,Biologie santé. Pathologie vasculaire et nutri...,Soutenance le 22/07/2022,Aix-Marseille,École Doctorale Sciences de la Vie et de la Sa...,Centre de recherche en cardiovasculaire et nu...,Patrice Darmon,Christine Feillet-Coudray,"Jérémie Talvas, Catherine Riva-Lavieille",Carence maternelle en vitamine D chez la souri...,La VD joue un rôle crucial dans le développeme...,https://theses.fr/2022AIXM0319,NaN,NaN
1,Shuaishuai Deng,Benjamin Loubet,Projet de thèse,Géosciences,Inscription en doctorat le 01/10/2023,université Paris-Saclay,École doctorale Sciences de l'environnement d'...,ECOSYS Écologie fonctionnelle et écotoxicolog...,NaN,NaN,NaN,Neglected impacts of air pollution and foliar ...,"Aerosols are deposited on leaf surfaces, which...",https://theses.fr/s367373,Université de Versailles-Saint-Quentin-en-Yve...,NaN


In [163]:
df_metadata.to_dict(orient= "records")

[{'Auteur / Autrice': 'Nicole Haroun',
  'Direction': 'Jean-François Landrier,Lourdes Mounien',
  'Type': 'Thèse de doctorat',
  'Discipline(s)': 'Biologie santé. Pathologie vasculaire et nutrition',
  'Date': 'Soutenance le  22/07/2022',
  'Etablissement(s)': 'Aix-Marseille',
  'Ecole(s) doctorale(s)': 'École Doctorale Sciences de la Vie et de la Santé (Marseille)',
  'Partenaire(s) de recherche': ' Centre de recherche en cardiovasculaire et nutrition (C2VN)  - Laboratoire Nutriments Lipides et Prévention des Maladies Métaboliques (Marseille) ',
  'Jury': ' Patrice Darmon',
  'Examinateurs / Examinatrices ': ' Christine Feillet-Coudray',
  'Rapporteurs / Rapporteuses ': ' Jérémie Talvas, Catherine Riva-Lavieille',
  'title': "Carence maternelle en vitamine D chez la souris : conséquences sur l'inflammation du tissu adipeux blanc et la stéatose hépatique dans la descendance",
  'content': "La VD joue un rôle crucial dans le développement périnatal. Mais la VDD est devenue un problème d

In [147]:
utilities_database.update_excel_query_search(pd.DataFrame(), df_metadata, "example.fr")

,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses,title,content,url_these,référent,Établissement co-accrédité,url_query
0,Nicole Haroun,"Jean-François Landrier,Lourdes Mounien",Thèse de doctorat,Biologie santé. Pathologie vasculaire et nutri...,Soutenance le 22/07/2022,Aix-Marseille,École Doctorale Sciences de la Vie et de la Sa...,Centre de recherche en cardiovasculaire et nu...,Patrice Darmon,Christine Feillet-Coudray,"Jérémie Talvas, Catherine Riva-Lavieille",Carence maternelle en vitamine D chez la souri...,La VD joue un rôle crucial dans le développeme...,https://theses.fr/2022AIXM0319,Missing value,Missing value,example.fr
1,Shuaishuai Deng,Benjamin Loubet,Projet de thèse,Géosciences,Inscription en doctorat le 01/10/2023,université Paris-Saclay,École doctorale Sciences de l'environnement d'...,ECOSYS Écologie fonctionnelle et écotoxicolog...,Missing value,Missing value,Missing value,Neglected impacts of air pollution and foliar ...,"Aerosols are deposited on leaf surfaces, which...",https://theses.fr/s367373,Université de Versailles-Saint-Quentin-en-Yve...,Missing value,example.fr
2,Sutha Leenawat,Flora Blanchon,Thèse de doctorat,Histoire de l'art et archéologie,Soutenance en 2001,Paris 4,Missing value,Missing value,Missing value,Missing value,Missing value,La peinture murale de l'école de Bangkok penda...,"Au XIX ième siècle, la Thai͏̈lande dut faire f...",https://theses.fr/2001PA040146,Missing value,Missing value,example.fr
3,Pierre Morel,"Pierre Bertrand,Etienne Gravier",Thèse de doctorat,Physique,Soutenance le 04/07/2008,Nancy 1,EMMA - Ecole Doctorale Energie - Mécanique - M...,LPMIA - Laboratoire de Physique des Milieux I...,Alain Ghizzo,"Dominique Escande, Giovanni Manfredi, Xavier ...",Missing value,Le modèle « water bag » appliqué aux équations...,Ce travail a porté sur l'étude des instabilité...,https://theses.fr/2008NAN10153,Missing value,Missing value,example.fr
4,Arunabha Kundu,Henri Delmas,Thèse de doctorat,Génie des procédés et environnement,Soutenance en 2004,"Toulouse, INPT",Missing value,Missing value,Missing value,Missing value,Missing value,"Réacteurs à trois phases : A, hydrodynamique e...",Two investigations on three phase reactors are...,https://theses.fr/2004INPT003G,Missing value,Missing value,example.fr
5,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,https://theses.fr/2016PA066125,Missing value,Missing value,example.fr
6,Laura Gouriou,Géraldine Gouhier,Projet de thèse,Chimie,Inscription en doctorat le 28/09/2022,Normandie,École doctorale normande de chimie (Caen),"réactivité et analyse (Mont-Saint-Aignan, Sei...",Missing value,Missing value,Missing value,New Ferromagnetic nanoparticles for water reme...,Missing value,https://theses.fr/s392384,Missing value,Université de Rouen Normandie (1966-....),example.fr
7,François Joseph Chatelon,Pierre Orenga,Thèse de doctorat,Mathématiques,Soutenance en 1996,Corte,Missing value,Missing value,Missing value,Missing value,Missing value,Analyse d'un problème de shallow water,Ce mémoire est consacré à l'étude d'un problèm...,https://theses.fr/1996CORT3018,Missing value,Missing value,example.fr
8,Randall Cherry,Geneviève Fabre,Thèse de doctorat,"Lettres, sciences humaines et sociales",Soutenance en 2002,Paris 7,Missing value,Missing value,Missing value,Missing value,Missing value,Mythologie de la voix noire : Ethel Waters et ...,Ce travail étudie la façon dont la chanteuse n...,https://theses.fr/2002PA070064,Missing value,Missing value,example.fr
9,Gilles Darblade,Alain-Yves Leroux,Thèse de doctorat,Mathématiques appliquées,Soutenance en 1997,Bordeaux 1,Missing value,Missing value,Missing value,Missing value,Missing value,Méthodes numériques et conditions aux limites ...,Les modeles shallow water multi

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
db = utilities_database.load_vector_store("./static/vector_store", embeddings=EMBEDDING)

In [10]:
RAG.document_into_context(db.similarity_search("test 1", 10))


' - Title document_0 : Missing value - Document_0 content : Missing value\n\n - Title document_1 : Synthèse et caractérisation de membranes hybrides pour la conduction des ions lithium, et application dans les batteries lithium-air à électrolyte aqueux - Document_1 content : La technologie lithium-air à électrolyte aqueux pourrait révolutionner le stockage de l\'énergie, mais la protection du lithium métallique par une vitrocéramique conductrice du lithium reste une limitation importante. Cela rend le système plus fragile, limite sa cyclabilité et augmente la chute ohmique. L\'objectif de ce travail a été de remplacer cette vitrocéramique par une membrane hybride réalisée par extrusion électro assistée ou electrospinning, qui combine des propriétés d\'étanchéité à l\'eau, de flexibilité et de conductivité du lithium. La conductivité ionique est apportée par la partie céramique, pour laquelle les matériaux Li1,4Al0,4Ti1,6(PO4)3 (LATP) et Li0,33La0,57TiO3 (LLTO) ont été étudiés. L\'étanc